## Setup

In [1]:
!pip install langchain
!pip install gpt4all
!pip install qdrant-client
!pip install sentence-transformers
!pip install pypdf


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/812.8 kB ? eta -:--:--
     ------------------------------------- 812.8/812.8 kB 25.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/394.9 kB ? eta -:--:--
     ---------------------------------------- 394.9/394.9 kB ? eta 0:00:00
     ---------------------------------------- 0.0/276.8 kB ? eta -:--:--
     ------------------------------------- 276.8/276.8 kB 16.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/87.5 kB ? eta -:--:--
     ---------------------------------------- 87.5/87.5 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
     ---------------------------------------- 2.1/2.1 MB 43.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
     ---------------------------------------- 1.9/1.9 MB 60.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/49.4 kB ? eta -:--:--
     ----------------


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/223.2 kB ? eta -:--:--
     ------------------------------------- 223.2/223.2 kB 13.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 1.1/1.1 MB 34.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/121.1 kB ? eta -:--:--
     ---------------------------------------- 121.1/121.1 kB ? eta 0:00:00
     ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
     ---------------------------------------  3.8/3.8 MB 121.9 MB/s eta 0:00:01
     ---------------------------------------- 3.8/3.8 MB 61.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
     ---------------------------------------- 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
     ------------------------------------- 413.4/413.4 kB 25.2 MB/s eta 0:00:00
     ---------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyppeteer 2.0.0 requires urllib3<2.0.0,>=1.25.8, but you have urllib3 2.2.1 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
     ------------------------------------- 163.3/163.3 kB 10.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/8.8 MB ? eta -:--:--
     --------------- ------------------------ 3.3/8.8 MB 104.0 MB/s eta 0:00:01
     ------------------------------- -------- 6.9/8.8 MB 88.2 MB/s eta 0:00:01
     ---------------------------------------  8.8/8.8 MB 79.9 MB/s eta 0:00:01
     ---------------------------------------- 8.8/8.8 MB 56.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
     ----------- --------------------------- 3.2/10.6 MB 101.7 MB/s eta 0:00:01
     --------------------------------- ----- 9.2/10.6 MB 117.5 MB/s eta 0:00:01
     -------------------------------------  10.6/10.6 MB 108.8 MB/s eta 0:00:01
     --------------------------------------- 10.6/10.6 MB 65.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/388.9 kB ? eta -:


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
     ------------------------------------  286.7/290.4 kB 17.3 MB/s eta 0:00:01
     -------------------------------------- 290.4/290.4 kB 4.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\rhihi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Set huggingface cache home (optional)

In [1]:
import os
cache_dir="huggingface/"
os.environ["HF_HOME"] = cache_dir

model_dir = ".\\models\\mistral-7b-instruct-v0.1.Q4_0.gguf"
os.environ["MODEL"] = model_dir

## Check Device (assuming no GPU is available)

I have a GPU available but for you it will print "cpu" if you don't.

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## Enable to see Logs while running

In [7]:
import logging

# Configure logging
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Create a logger
logger = logging.getLogger(__name__)

## Load Data, preprocessing and Creating Qdrant Index

In [10]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
import re

def preprocess_text(text):
    text_lower = text.lower()
    text_no_punctuation = re.sub(r'[^\w\s\$\%\.\,\"\'\!\?\(\)]', '', text_lower)
    text_normalized_tabs = re.sub(r'(\t)+', '', text_no_punctuation)
    return text_normalized_tabs

# loader = PyPDFLoader("data/msft_annual_2023_report.pdf")
loader = PyPDFLoader("data/Recipes.pdf")
# loader = TextLoader(sample_texts)
# loader = WebBaseLoader("https://cleartax.in/s/top-performing-nps-schemes")
documents = loader.load()
for x in range(len(documents)):
    # do preprocessing
    documents[x].page_content=preprocess_text(documents[x].page_content)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0,separator="\n")
docs = text_splitter.split_documents(documents)

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5", 
                                   model_kwargs = {'device': "cpu"})  # forcefully setting device as cpu
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="msft_data",
    force_recreate=True
)

2024-04-08 17:27:10,670 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
2024-04-08 17:27:10,782 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/modules.json HTTP/1.1" 200 0
2024-04-08 17:27:10,965 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/config_sentence_transformers.json HTTP/1.1" 200 0
2024-04-08 17:27:11,085 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/README.md HTTP/1.1" 200 0
2024-04-08 17:27:11,185 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/modules.json HTTP/1.1" 200 0
2024-04-08 17:27:11,297 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /BAAI/bge-large-en-v1.5/resolve/main/sentence_bert_config.json HTTP/1.1" 200 0
2024-04-08 17:27:11,396 - urllib3.connecti

## Run if not able to download model using GPT4All directly.

In [ ]:
# # download model

# !mkdir models
# !wget https://gpt4all.io/models/gguf/mistral-7b-instruct-v0.1.Q4_0.gguf -O models/mistral-7b-instruct-v0.1.Q4_0.gguf

## Directly calling GPT4All module 

In [11]:
from gpt4all import GPT4All
model = GPT4All(model_name="F:\RAG_PFE\RAG\models\mistral-7b-instruct-v0.1.Q4_0.gguf",                             n_threads = 8,
                             allow_download=False) # set allow_download as true to fetch it.

In [12]:
prompt = "Eiffel tower location:"
print(model.generate(prompt, max_tokens=20))


Champ de Mars, 5 Avenue Anatole France, Paris, France.


## Creating prompts, tuning parameters and creating Chain using Langchain

In [13]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import PromptTemplate, LLMChain
from langchain_community.llms import GPT4All

# template = '''[INST]: You are a financial expert analyst bot, below presents a context from which the a question will be asked, give your valuable insights as well.[\INST]\n
# Context: {context}.\n
# Question: {question}\n
# Answer: '''


# You are a chef with extensive knowledge of various recipes. 
# Based on the context provided, which could either be specific ingredients the 
# user has on hand or a type of dish the user desires, offer a suitable recipe that matches the request.

 

# template = '''[INST]:As a chef renowned for your vast repertoire of recipes, utilize the provided context
# to craft a recipe. Whether the inquiry involves specific ingredients available to the 
# user or a particular dish they're craving, draw upon your culinary expertise to recommend
# a recipe that aligns with their needs. [\INST]\n

template = '''[INST]: As a chef renowned for your vast culinary knowledge and experience, use the context
provided to deliver a comprehensive recipe. This entails not only guiding on the preparation
of a specific dish with the available ingredients but also enriching the recipe with nutritional
insights such as calorie count, protein, fat, and carbohydrate content. Whether the query
involves utilizing specific ingredients at hand or crafting a dish to satisfy a particular
craving, employ your expertise to recommend a balanced, nutritious recipe that caters to 
the user's dietary preferences and goals. Include all necessary steps for preparation, 
cooking tips, and garnishing advice to ensure a delightful culinary experience. Ensure 
the recipe is clear, accessible, and tailored to the user's needs, providing a wholesome
meal that is both satisfying and health-conscious. [\INST]\n


Context: {context}.\n
Question: {question}\n
Answer: '''
rag_prompt = PromptTemplate(template=template, input_variables=["context","question"])

callbacks = [StreamingStdOutCallbackHandler()]

llm = GPT4All(
            model="F:\RAG_PFE\RAG\models\mistral-7b-instruct-v0.1.Q4_0.gguf",
            max_tokens=50,
            n_threads=8, 
            temp=0.3,
            top_p=0.2,
            top_k=40,
            n_batch=8,
            seed=100,
            allow_download=False,  # Prevent downloading
            verbose=True)

llm_chain = LLMChain(prompt=rag_prompt, llm=llm, verbose=True)

## Define format_docs for formatting context candidates.

In [14]:
def format_docs(query):
    found_docs = qdrant.similarity_search_with_score(query,k=1)
    return "\n\n".join(doc[0].page_content for doc in found_docs)

## Run Queries

In [15]:
%%time
query = "What can I make with chicken, tomatoes, and rice?"
context = "The user has chicken, tomatoes, and rice available and is looking for a recipe that uses these ingredients."
resp = llm_chain.invoke(
    input={"question": query,
           "context": context
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]: As a chef renowned for your vast culinary knowledge and experience, use the context
provided to deliver a comprehensive recipe. This entails not only guiding on the preparation
of a specific dish with the available ingredients but also enriching the recipe with nutritional
insights such as calorie count, protein, fat, and carbohydrate content. Whether the query
involves utilizing specific ingredients at hand or crafting a dish to satisfy a particular
craving, employ your expertise to recommend a balanced, nutritious recipe that caters to 
the user's dietary preferences and goals. Include all necessary steps for preparation, 
cooking tips, and garnishing advice to ensure a delightful culinary experience. Ensure 
the recipe is clear, accessible, and tailored to the user's needs, providing a wholesome
meal that is both satisfying and health-conscious. [\INST]



Context: The user has chicken, tomatoes, and rice available 

In [28]:
%%time
query = "What can I make with chicken, tomatoes, and rice?"
# context = "The user has chicken, tomatoes, and rice available and is looking for a recipe that uses these ingredients."
resp = llm_chain.invoke(
    input={"question": query,
           "context": format_docs(query)
          }
)
print(resp['text'])



> Entering new LLMChain chain...
Prompt after formatting:
[INST]:As a chef renowned for your vast repertoire of recipes, utilize the provided context
to craft a recipe. Whether the inquiry involves specific ingredients available to the 
user or a particular dish they're craving, draw upon your culinary expertise to recommend
a recipe that aligns with their needs. [\INST]


Context: thoroughly. once cooked (approximately 10 minutes, place to the side)
3.in your cooked chicken bowl, add your sauce to the chicken. 28 g of fatfree cheese, 60g of nonfat
plain greek yogurt, one serving of e cheese, 60 g of nonfat plain greek yogurt, one serving of 60 g
of lowcal alfredo sauce, add a teaspoon of minced garlic as well as either ranch seasoning or
parmesan powder, a hint of lemon, and diced parsley. also, add salt and pepper to your taste.
4.empty contents into a mediumsized pan and cook down until you notice the cheese melting into
the sauce. add your noodles (strain the water first) to your

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x0000021004A7FD90>
Traceback (most recent call last):
  File "C:\Users\rhihi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gpt4all\_pyllmodel.py", line 526, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 


## Other Queries ideas

query = 